In [7]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

In [8]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-06-17 14:52:01--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  1.58MB/s    in 0.6s    

2021-06-17 14:52:03 (1.58 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [9]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [11]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [15]:
print(df.count())
df.describe()

9002021


DataFrame[summary: string, marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: string, helpful_votes: string, total_votes: string, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [18]:
from pyspark.sql.types import IntegerType
updated_df=df.withColumn('star_rating',df['star_rating'].cast(IntegerType()))
updated_df=df.withColumn('helpful_votes',df['helpful_votes'].cast(IntegerType()))
updated_df=df.withColumn('total_votes',df['total_votes'].cast(IntegerType()))
updated_df.printSchema

<bound method DataFrame.printSchema of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: string, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]>

In [19]:
cleaned_df=updated_df.dropna()
print(cleaned_df.count())

9001052


In [20]:
df=cleaned_df.dropDuplicates()
df.count()

9001052

### Create DataFrames to match tables

In [26]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [28]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   49070323|             3|
|    4861668|             3|
|   44920760|             1|
|   33413426|             1|
|   32971810|             1|
|   24086603|            10|
|   31680089|             4|
|   25845557|             2|
|   47638296|             7|
|   47485107|             1|
|   45970346|             2|
|   20284767|             8|
|   39069721|             4|
|   15532706|             1|
|   28891561|             3|
|   43614803|             1|
|   14053649|             3|
|   52767441|             1|
|   20843308|             1|
|   29385897|             9|
+-----------+--------------+
only showing top 20 rows



In [21]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).distinct()
products_df.count()

906482

In [27]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id', 'customer_id', 'product_id', 'product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10048FCFWVJOH|   34974850|B002OL2MU4|     471222439| 2011-04-07|
|R1008YTRJEH6U7|   30903818|B00LAHPLTW|     355012266| 2014-12-15|
|R100F3WWLNCX5S|   51491202|B003T91K94|     816619821| 2014-07-31|
|R100IJSLJ25VPG|    2572903|B00ENI0JM4|     653653620| 2014-11-30|
|R100JPDEL3NLYL|   37174514|B00EF1OGOG|     558284639| 2014-07-01|
|R100KWGZ8UZDA0|   13654097|B0079A29L2|     198567336| 2013-04-21|
|R100LHWI1PCRDO|   23612091|B007AQF0WK|      19433141| 2014-12-21|
|R100MJFI7QHR78|   44895028|B00PNUBGGY|     740916364| 2015-04-02|
|R100MKHZ325MI7|   35797687|B00DEKJEAA|      24966059| 2014-06-11|
|R100U625Y5RVRJ|   25608655|B00CI4VKB4|     689535231| 2013-11-21|
|R100USE1L3IPHJ|   20009407|B00JFNICH6|     102684500| 2015-08-27|
|R101095URF10PU|   33342037|B0073ZPI0W|     123057041| 2012-02

In [34]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df=vine_df.withColumn('star_rating',df['star_rating'].cast(IntegerType()))
# vine_df.printSchema
# vine_df.show(5)

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [22]:
# Configure settings for RDS
from getpass import getpass
password=getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cd5tg7trpxyp.us-east-2.rds.amazonaws.com:5432/Amazon_Vines"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [36]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [37]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [35]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)